# Importancia de la permutación

## EJERCICIOS

### Introducción

Vamos a pensar y calcular la Importancia de la Permutación con una muestra de datos de la competicion [Predicción de tarifas de taxi](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction).

Por ahora no nos centraremos en la exploración de datos o en la construcción de modelos. Simplemente ejecuta la celda de abajo para
- Cargar los datos.
- Dividir los datos en entrenamiento y validación.
- Contruir un modelo que predice las tarifas de los taxis.
- Imprimir unas cuantas filas para revisarlas.

In [1]:
# Carga de datos, división, modelado y EDA
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = pd.read_csv('./data/new-york-city-taxi-fare-prediction/train.csv', nrows=50000)

# Eliminamos datos con coordenadas con valores atípicos extremos o tarifas negativas
data = data.query('pickup_latitude > 40.7 and pickup_latitude < 40.8 and ' +
                  'dropoff_latitude > 40.7 and dropoff_latitude < 40.8 and ' +
                  'pickup_longitude > -74 and pickup_longitude < -73.9 and ' +
                  'dropoff_longitude > -74 and dropoff_longitude < -73.9 and ' +
                  'fare_amount > 0'
                  )

y = data.fare_amount

base_features = ['pickup_longitude',
                 'pickup_latitude',
                 'dropoff_longitude',
                 'dropoff_latitude',
                 'passenger_count']

X = data[base_features]


train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
first_model = RandomForestRegressor(n_estimators=30, random_state=1).fit(train_X, train_y)

# Mostramos datos
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1


In [2]:
train_X.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,23466.000000,23466.000000,23466.000000,23466.000000,23466.000000
mean,-73.976827,40.756931,-73.975359,40.757434,1.662320
std,0.014625,0.018206,0.015930,0.018659,1.290729
min,-73.999999,40.700013,-73.999999,40.700020,0.000000
25%,-73.987964,40.744901,-73.987143,40.745756,1.000000
50%,-73.979629,40.758076,-73.978588,40.758542,1.000000
75%,-73.967797,40.769602,-73.966459,40.770406,2.000000
max,-73.900062,40.799952,-73.900062,40.799999,6.000000


In [3]:
train_y.describe()

count    23466.000000
mean         8.472539
std          4.609747
min          0.010000
25%          5.500000
50%          7.500000
75%         10.100000
max        165.000000
Name: fare_amount, dtype: float64

## Pregunta 1

El primer modelo usa las siguientes características
- pickup_longitude (longitud de recogida)
- pickup_latitude (latitud de recogida)
- dropoff_longitude (longitud de bajada)
- dropoff_latitude (latitud de bajada)
- passenger_count (número de pasajeros)

Antes de ejecutar cualquier código... ¿Qué variables parecen potencialmente útiles para predecir las tarifas de taxi? ¿Piensas que la Importancia de la Permutación identificará necesariamente estas características como importantes?

***Solución***: Sería útil saber si los taxis de la Ciudad de Nueva York varían los precios según la cantidad de pasajeros que tienen. La mayoría de los lugares no cambian las tarifas según el número de pasajeros. Si asumimos que en la ciudad de Nueva York es igual, solo deberían importar las 4 características principales enumeradas. A primera vista, parece que todos esos deberían importar por igual.

## Pregunta 2

Crea un objeto `PermutationImportance` llamado `perm` para mostrar las importancias del primer modelo. Entrénalo con los datos apropiados y muestra sus pesos.

In [4]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(first_model, random_state=1).fit(val_X, val_y)

eli5.show_weights(perm, feature_names = val_X.columns.tolist())

Weight,Feature
0.8474 ± 0.0185,dropoff_latitude
0.8267 ± 0.0263,pickup_latitude
0.6229 ± 0.0559,pickup_longitude
0.5384 ± 0.0305,dropoff_longitude
-0.0029 ± 0.0020,passenger_count


## Pregunta 3
Antes de ver estos resultados, podríamos haber esperado que cada una de las 4 características direccionales fueran igualmente importantes. Pero, en promedio, las características de latitud son más importantes que las características de longititud. ¿Se puede formular alguna hipótesis para esto?

Después de haberlo pensado, revisa aquí algunas explicaciones posibles:

***Solución***: 
1. El viaje puede tender a tener mayores distancias de latitud que distancias de longitud. Si los valores de longitudes estuvieran generalmente más juntos, mezclarlos no importaría tanto.
2. Diferentes zonas de la ciudad pueden tener diferentes precios (por ejemplo, precio por milla), y los precios pueden variar más por latitud que por longitud.
3. Los peajes pueden ser mayores en las carreteras que van hacia el Norte <-> Sur (cambio de latitud) que en las carreteras que van hacia el Este <-> Oeste (cambio de longitud). Por lo tanto, la latitud tendría un efecto mayor en la predicción porque contiene mayor número de peajes.

## Pregunta 4

Sin un conocimiento detallado de la ciudad de Nueva York, es difícil descartar la mayoría de las hipótesis acerca de por qué las características de latitud son más importantes que la longitud.

Un buen paso siguiente es desenmarañar el efecto de estar en ciertas partes de la ciudad del efecto de la distancia total recorrida.

El siguiente código crea nuevas características para la distancia longitudinal y latitudinal. Luego construye un modelo que agrega estas nuevas características a las que ya teníamos.

Completa las dos líneas de código para calcular y mostrar los pesos de importancia con este nuevo conjunto de características.

In [5]:
# Crea las nuevas características
data['abs_lon_change'] = abs(data.dropoff_longitude - data.pickup_longitude)
data['abs_lat_change'] = abs(data.dropoff_latitude - data.pickup_latitude)

features_2  = ['pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'abs_lat_change',
               'abs_lon_change']

X = data[features_2]
new_train_X, new_val_X, new_train_y, new_val_y = train_test_split(X, y, random_state=1)
second_model = RandomForestRegressor(n_estimators=30, random_state=1).fit(new_train_X, new_train_y)

# Crea un objeto PermutationImportance en un segundo modelo y lo entrena en new_val_X y new_val_y
# Usamos random_state de 1 para reproducir los resultados que coincidan con la solución esperada
perm2 = PermutationImportance(second_model, random_state=1).fit(new_val_X,new_val_y)

# Muestra los pesos para la Importancia de la Permutación que acabamos de calcular
eli5.show_weights(perm2, feature_names = features_2)

Weight,Feature
0.5783 ± 0.0295,abs_lat_change
0.4467 ± 0.0509,abs_lon_change
0.0858 ± 0.0333,pickup_latitude
0.0735 ± 0.0101,dropoff_longitude
0.0733 ± 0.0113,dropoff_latitude
0.0613 ± 0.0063,pickup_longitude


¿Cómo interpretarías estas puntuaciones de importancia? La distancia recorrida parece mucho más importante que cualquier efecto de ubicación.

Pero la ubicación todavía afecta a las predicciones del modelo y la ubicación de entrega ahora importa un poco más que la ubicación de recogida. ¿Tienes alguna hipótesis de por qué podría ser esto? Las técnicas utilizadas más adelante en el curso nos ayudarán a sumergirnos más en esto.

## Pregunta 5

Un colega observa que los valores para `abs_lon_change` y` abs_lat_change` son bastante pequeños (todos los valores están entre -0.1 y 0.1), mientras que otras variables tienen valores más grandes. ¿Crees que esto podría explicar por qué esas coordenadas tenían valores de importancia de permutación más grandes en este caso?

Considera una alternativa donde creaste y usaste una característica que era 100 veces más grande para estas características, y usaste esa característica más grande para los cálculos de entrenamiento e importancia. ¿Cambiaría esto los valores de importancia de permutación obtenidos?

¿Por qué y por qué no?

***Solución***: La escala de características no afecta a la importancia de la permutación *per se*. La única razón por la que el cambio de escala de una función afectaría a la importancia de la permutación es indirectamente, si el cambio de escala ayuda o perjudica la capacidad del método de aprendizaje en particular que estamos utilizando para hacer uso de esa función. Eso no sucederá con los modelos basados en árboles, como el Random Forest que se usa aquí. Si estás familiarizado con la Regresión de Ridge, es posible que pueda pensar en cómo se vería afectada. Dicho esto, las características de cambio absoluto tienen una gran importancia porque capturan la distancia total recorrida, que es el determinante principal de las tarifas de taxi ... No es un artefacto de la magnitud de la característica.

## Pregunta 6

Has visto que la importancia de la característica para la distancia latitudinal es mayor que la importancia de la distancia longitudinal. A partir de esto, ¿podemos concluir si viajar una distancia latitudinal fija tiende a ser más costoso que viajar la misma distancia longitudinal?

¿Por qué o por qué no? Comprueba tu respuesta a continuación.

***Solución***: No podemos decir por los resultados de la importancia de la permutación si viajar una distancia latitudinal fija es más o menos costoso que viajar la misma distancia longitudinal. Las posibles razones de por qué las características de latitud son más importantes que las de longitud son las siguientes:

1. Las distancias latitudinales en el conjunto de datos tienden a ser mayores.
2. Es más caro recorrer una distancia latitudinal fija.
3. Las dos anteriores

Si los valores de `abs_lon_change` fueran muy pequeños, los valores largos podrían ser menos importantes para el modelo, incluso si el costo por milla de viaje en esa dirección fuera alto.

## Felicidades

La importancia de la permutación es útil para depurar, comprender su modelo y comunicar una descripción general de alto nivel de su modelo. A continuación, se mostrarán gráficos de dependencia parciales, que le ayudarán a ver cómo las características individuales afectan las predicciones.